In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.query_pipeline import QueryPipeline, InputComponent, FnComponent
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.schema import NodeWithScore
from typing import Dict, Any, List
from abc import ABC, abstractmethod
from llama_index.core import VectorStoreIndex
from llama_index.core.prompts import PromptTemplate
from llama_index.core.vector_stores import FilterCondition, FilterOperator
from llama_index.core.vector_stores.types import (
    ExactMatchFilter,
    MetadataFilter,
    MetadataFilters,
)

In [ ]:
embed_model = OpenAIEmbedding(
    model="text-embedding-ada-002", 
    timeout=60,
    api_key="sk-xxxx"
)
llm_model = OpenAI(model="gpt-3.5-turbo",timeout=60,api_key="sk-xxxx")

In [ ]:
import qdrant_client

client = qdrant_client.QdrantClient(
    "http://localhost:6333"
)

vector_store = QdrantVectorStore(
    client=client, collection_name="qdr"
)

In [ ]:
filters={
    # Depending on the mode ('collection' or 'file'), index is either the collection_id or the token
    "index": "667843591624d2699fdc8ce2"
}

llama_index_filters = MetadataFilters(
    filters=[
        MetadataFilter(key=key, value=value)
        for key, value in filters.items()
    ],
    condition=FilterCondition.AND,
)
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model,
    filters=llama_index_filters,
)

In [ ]:
prompt="""
        Tu es un assistant documentaire. Ta missions est de répondre à des questions à partir uniquement du contexte suivant. Si tu ne connais pas l'information, tu diras que tu ne connais pas la réponse. Ta réponse sera concise. Tu utiliseras uniquement l'information pertiente. 
        Ta réponse devra obligatoirement être en français, c'est un point très important.
        Voici un exemple de réponse :
        - L'IA générative se base sur l'architecture transformer développée par Google.
        
        Contexte : 
        {context_str}

        Question :
        {query_str}  

        Répond à la question en te basant sur le contexte ci-dessus
"""

In [ ]:
llama_index_filters = MetadataFilters(
    filters=[
        MetadataFilter(key=key, value=value)
        for key, value in filters.items()
    ],
    condition=FilterCondition.AND,
)
query_engine = index.as_query_engine(
    llm=llm_model,
    filters=llama_index_filters,
    similarity_top_k=5,
    streaming=False,
)
query_engine.update_prompts(
            {
                "response_synthesizer:text_qa_template": PromptTemplate(
                    prompt
                )
            }
        )

In [ ]:
result = query_engine.query("C'est quoi un AAE ?")

In [ ]:
result